## Decorators

Functions that wrap around another function or method, allowing you to execute code before and/or after the wrapped function is called, or even replace the wrapped function entirely.

In [6]:
# Here, say_hello() and be_awesome() are regular functions that expect a name given as a string. 
# The greet_bob() function, however, expects a function as its argument. 
# You can, for example, pass it the say_hello() or the be_awesome() function.

def say_hello(name):
    return f"Hello {name}"

def be_awesome(name):
    return f"Yo {name}, together we're the awesomest!"

def greet_bob(greeter_func):
    return greeter_func("Bob")

greet_bob(say_hello)
greet_bob(be_awesome)


"Yo Bob, together we're the awesomest!"

* Notice that `great_bob` is being called (i.e. wirtten with parenthesis)
* On the other hand `say_hello` or `be awesome` are not with parenthesis. Therefore, they are not being called. <u>Only a reference of the function is being passed</u>

In [12]:
def decorator(func):
    def wrapper():
        print("Something is happening before the function is called.")
        func()
        print("Something is happening after the function is called.")
    return wrapper

def say_whee():
    print("Whee!")

say_whee = decorator(say_whee)  # Here we call the function decorator, and then pass a reference to say-whee to the wrapper

# Just to show it is a decorator
print(say_whee)

# Call the function
say_whee()

<function decorator.<locals>.wrapper at 0x7fe6d7db64c0>
Something is happening before the function is called.
Whee!
Something is happening after the function is called.


##### Syntactic Sugar for Decorators

In example above, we had to type `say_whee` three times. We can save up time and use the `pie syntax` symbol, or `@` symbol

In [13]:
def decorator(func):
    def wrapper():
        print("Something is happening before the function is called.")
        func()
        print("Something is happening after the function is called.")
    return wrapper

@decorator
def say_whee():
    print("Whee!")

***<u>So, @decorator is just a shorter way of saying say_whee = decorator(say_whee). It’s how you apply a decorator to a function.</u>***

## Decorating Functions With Arguments

Below I pass function `greet` with arguments to our decorator

In [16]:
@decorator
def greet(name):
    print(f"Hello {name}")

try:
    greet('Juan')
except TypeError:
    print('need to pass arguments to wrapper')

need to pass arguments to wrapper


***Passing arguments to `wrapper` can be problematic, because maybe not all functions that are being decorated with a single decorator have the same arguments. To solve it, use **args and **kwargs***

In [17]:
def decorator(func):
    def wrapper(*args, **kwargs):
        print("Something is happening before the function is called.")
        func(*args, **kwargs)
        print("Something is happening after the function is called.")
    return wrapper

@decorator
def greet(name):
    print(f"Hello {name}")

greet('Juan')

Something is happening before the function is called.
Hello Juan
Something is happening after the function is called.


##### Another example...

In [20]:
def logger_on(afunction):
    def wrapper(*args, **kwargs):
        print(f'executing function {afunction.__name__}')
        print(f'The parameters are args = {args} and kwargs = {kwargs}')
        result = afunction(*args, **kwargs)
        return result
    return wrapper

@logger_on
def addition_funny(num1, num2):
    return num1 + num2

print(addition_funny(num1=2,num2=3))


executing function addition_funny
The parameters are args = () and kwargs = {'num1': 2, 'num2': 3}
5


##### Same example but without explicit use of decorator

In [21]:
def logger_on(afunction):
    def wrapper(*args, **kwargs):
        print(f'executing function {afunction.__name__}')
        print(f'The parameters are args = {args} and kwargs = {kwargs}')
        result = afunction(*args, **kwargs)
        return result
    return wrapper

def addition_funny(num1, num2):
    return num1 + num2

print(logger_on(addition_funny)(num1=2, num2=3))  # Without decorator

executing function addition_funny
The parameters are args = () and kwargs = {'num1': 2, 'num2': 3}
5


## Closures
A closure in programming refers to a function that captures and retains the environment in which it was created, including the variables and bindings that were in scope at the time of its creation. This allows the function to access and manipulate those variables even after the scope in which they were defined has exited.

In simpler terms, ***<u>a closure is a function that remembers the environment in which it was created, and it can access variables from that environment even after the enclosing function has finished executing.</u>***

In [23]:
def outer_function(x):
    def inner_function(y):
        return x + y
    return inner_function

closure = outer_function(10)
result = closure(5)  

print(result)

15


Another example...

In [30]:
def generate_power(exponent):   # CLOSURE FACTORY FUNCTION. This means that it creates a new closure each time it’s called 
    def power(base):   # INNER FUNCTION that takes a single argument
        return base ** exponent
    return power

# When you call generate_power():
#   1. Define a new instance of power(), which takes a single argument base.
#   2. Take a snapshot of the surrounding state of power(), which includes exponent with its current value.
#   3. Return power() along with its whole surrounding state.


raise_two = generate_power(2)

raise_three = generate_power(3)

# raise_two() remembers that exponent=2
print(raise_two(4))
print(raise_two(6))

# raise_three() remembers that exponent=3
print(raise_three(4))
print(raise_three(6))

16
36
64
216


##### IMPORTANT:

***Closure can modify their enclosing state by using mutable objects, such as dictionaries, sets, or lists***

In [39]:
def mean():
    sample = []
    def inner_mean(number):
        sample.append(number)
        return sum(sample)/len(sample)
    return inner_mean


sample_mean = mean()

print(sample_mean(100))   # Add 100. Mean should be sum([100])/len([100]) = 100

print(sample_mean(200))   # Add 200. Mean should be sum([100,200])/len([100,200]) = 150

print(sample_mean(75))   # Add 75. Mean should be sum([100,200,75])/len([100,200,75]) = 125

100.0
150.0
125.0


In [41]:
def memoize(afunction):
    cache = {}
    def wrapper(*args):
        if args not in cache:
            cache[args] = afunction(*args)
        print(cache)
        return cache[args]
    return wrapper

@memoize
def fibonacci(n):
    if n < 2:
        return n
    else:
        return fibonacci(n-1) + fibonacci(n-2)
    
fibonacci(9)

{(1,): 1}
{(1,): 1, (0,): 0}
{(1,): 1, (0,): 0, (2,): 1}
{(1,): 1, (0,): 0, (2,): 1}
{(1,): 1, (0,): 0, (2,): 1, (3,): 2}
{(1,): 1, (0,): 0, (2,): 1, (3,): 2}
{(1,): 1, (0,): 0, (2,): 1, (3,): 2, (4,): 3}
{(1,): 1, (0,): 0, (2,): 1, (3,): 2, (4,): 3}
{(1,): 1, (0,): 0, (2,): 1, (3,): 2, (4,): 3, (5,): 5}
{(1,): 1, (0,): 0, (2,): 1, (3,): 2, (4,): 3, (5,): 5}
{(1,): 1, (0,): 0, (2,): 1, (3,): 2, (4,): 3, (5,): 5, (6,): 8}
{(1,): 1, (0,): 0, (2,): 1, (3,): 2, (4,): 3, (5,): 5, (6,): 8}
{(1,): 1, (0,): 0, (2,): 1, (3,): 2, (4,): 3, (5,): 5, (6,): 8, (7,): 13}
{(1,): 1, (0,): 0, (2,): 1, (3,): 2, (4,): 3, (5,): 5, (6,): 8, (7,): 13}
{(1,): 1, (0,): 0, (2,): 1, (3,): 2, (4,): 3, (5,): 5, (6,): 8, (7,): 13, (8,): 21}
{(1,): 1, (0,): 0, (2,): 1, (3,): 2, (4,): 3, (5,): 5, (6,): 8, (7,): 13, (8,): 21}
{(1,): 1, (0,): 0, (2,): 1, (3,): 2, (4,): 3, (5,): 5, (6,): 8, (7,): 13, (8,): 21, (9,): 34}


34

In [42]:
fibonacci(4)

{(1,): 1, (0,): 0, (2,): 1, (3,): 2, (4,): 3, (5,): 5, (6,): 8, (7,): 13, (8,): 21, (9,): 34}


3

In [43]:
fibonacci(2)

{(1,): 1, (0,): 0, (2,): 1, (3,): 2, (4,): 3, (5,): 5, (6,): 8, (7,): 13, (8,): 21, (9,): 34}


1

In [44]:
fibonacci(15)

{(1,): 1, (0,): 0, (2,): 1, (3,): 2, (4,): 3, (5,): 5, (6,): 8, (7,): 13, (8,): 21, (9,): 34}
{(1,): 1, (0,): 0, (2,): 1, (3,): 2, (4,): 3, (5,): 5, (6,): 8, (7,): 13, (8,): 21, (9,): 34}
{(1,): 1, (0,): 0, (2,): 1, (3,): 2, (4,): 3, (5,): 5, (6,): 8, (7,): 13, (8,): 21, (9,): 34, (10,): 55}
{(1,): 1, (0,): 0, (2,): 1, (3,): 2, (4,): 3, (5,): 5, (6,): 8, (7,): 13, (8,): 21, (9,): 34, (10,): 55}
{(1,): 1, (0,): 0, (2,): 1, (3,): 2, (4,): 3, (5,): 5, (6,): 8, (7,): 13, (8,): 21, (9,): 34, (10,): 55, (11,): 89}
{(1,): 1, (0,): 0, (2,): 1, (3,): 2, (4,): 3, (5,): 5, (6,): 8, (7,): 13, (8,): 21, (9,): 34, (10,): 55, (11,): 89}
{(1,): 1, (0,): 0, (2,): 1, (3,): 2, (4,): 3, (5,): 5, (6,): 8, (7,): 13, (8,): 21, (9,): 34, (10,): 55, (11,): 89, (12,): 144}
{(1,): 1, (0,): 0, (2,): 1, (3,): 2, (4,): 3, (5,): 5, (6,): 8, (7,): 13, (8,): 21, (9,): 34, (10,): 55, (11,): 89, (12,): 144}
{(1,): 1, (0,): 0, (2,): 1, (3,): 2, (4,): 3, (5,): 5, (6,): 8, (7,): 13, (8,): 21, (9,): 34, (10,): 55, (11,): 89

610